In [9]:
import pandas as pd
import pulp as p
import pickle

In [10]:
bidDetails = pd.read_csv('./output/bids_details.csv')
bidDetails

,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice
0,1899326,432118,SUPPLIERRPC04,100,85,850
1,1899326,432118,SUPPLIERRPC06,100,92,920
2,1899326,432118,SUPPLIERRPC05,100,95,950


In [11]:
# https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range
minBidPrice = bidDetails['Quoted_Price'].min()
print(minBidPrice)
maxBidPrice = bidDetails['Quoted_Price'].max()
print(maxBidPrice)

85
95


In [12]:
minRange = 1.0
maxRange = 10.0
bidDetails['ScaledQuotedPrice'] = round((bidDetails['Quoted_Price'] - minBidPrice)/(maxBidPrice - minBidPrice) * (maxRange - minRange) + minRange, 2)

In [13]:
modelFile = open('./output/selected_ai_ml_model.mdl', 'rb')
model = pickle.load(modelFile)
modelFile.close()

model

DecisionTreeRegressor()

In [14]:
supplierMapFile = open('./output/suppliermap.dict', 'rb')
supplierMap = pickle.load(supplierMapFile)
supplierMapFile.close()
supplierMap

{'SUPPLIERRPC04': 10001, 'SUPPLIERRPC06': 10002, 'SUPPLIERRPC05': 10003}

In [15]:
for index, row in bidDetails.iterrows():
    commodityId = row['CommodityId']
    supplierId = supplierMap[row['SupplierId']]
    price = row['Quoted_Price']
    print("(" + str(row['SupplierId']) + ", " + str(commodityId) + ", " + str(price) + ")")
    
    bidData = pd.DataFrame([[supplierId, commodityId, price]])
    predict_bid = model.predict(bidData)
    
    quality = predict_bid[0][0]
    timeliness = predict_bid[0][1]
    
    print("Quality = " + str(quality) + ", Timeliness = " + str(timeliness))
    print("------------------------------------------------------------------")

    bidDetails.at[index,'Quality'] = quality
    bidDetails.at[index, 'Timeliness'] = timeliness

bidDetails

(SUPPLIERRPC04, 432118, 85)
Quality = 5.0, Timeliness = 9.0
------------------------------------------------------------------
(SUPPLIERRPC06, 432118, 92)
Quality = 5.0, Timeliness = 9.0
------------------------------------------------------------------
(SUPPLIERRPC05, 432118, 95)
Quality = 9.0, Timeliness = 6.0
------------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice,ScaledQuotedPrice,Quality,Timeliness
0,1899326,432118,SUPPLIERRPC04,100,85,850,1.0,5.0,9.0
1,1899326,432118,SUPPLIERRPC06,100,92,920,7.3,5.0,9.0
2,1899326,432118,SUPPLIERRPC05,100,95,950,10.0,9.0,6.0


In [16]:
bidDetails['ScaledQuotedPrice'] = (10 - bidDetails['ScaledQuotedPrice']) + 1
bidDetails

,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice,ScaledQuotedPrice,Quality,Timeliness
0,1899326,432118,SUPPLIERRPC04,100,85,850,10.0,5.0,9.0
1,1899326,432118,SUPPLIERRPC06,100,92,920,3.7,5.0,9.0
2,1899326,432118,SUPPLIERRPC05,100,95,950,1.0,9.0,6.0


In [17]:
bidDetails.to_csv('./output/bidding_supplier_characteristics_prediction.csv')
